In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

# 여러 메시지를 조합해서 "대화 프롬프트"를 만드는 클래스 불러오기
# - ChatMessagePromptTemplate : 개별 메시지를 만드는 도구
# - ChatPromptTemplate        : 여러 메시지를 합쳐 하나의 대화 흐름을 만드는 도구

chat_prompt = ChatPromptTemplate.from_messages(
    [
    # system 메시지 : 모델에게 기본적인 성격, 역할을 알려줌
    ("system", "당신의 MBTI 는 {mbti} 인, {style} 스타일의 {gender}야. 너의 스타일에 맞게 잘 답변해줘"),

    # human 메시지 : 사람이 모델에게 보낸 말
    ("human", "안녕하세요"),

    # ai 메시지 : 모델이 이전 대화에서 했던 답변을 미리 설정
    ("ai", "안녕하세요, 연애 상담해드릴까요"),

    # human 메시지 : 새로운 질문 (변수 {question}을 나중에 입력으로 교체)
    ("human", "{question} 에 대해 너의 스타일대로 답변해줘.")
    ]
)

# 체인(chain) 구성 : "프롬프트 → 모델 → 출력 파서"
# 1) chat_prompt : 대화 프롬프트 만들기
# 2) model       : LLM(모델)이 답변 생성
# 3) StrOutputParser() : 모델 답변 중에서 "텍스트만" 추출
chain = chat_prompt | model | StrOutputParser()
chain
# chain 객체는 이제 질문만 넣어주면
# → 프롬프트를 완성하고
# → 모델이 답변을 하고
# → 텍스트만 꺼내주는 일련의 과정을 자동으로 처리함


ChatPromptTemplate(input_variables=['gender', 'mbti', 'question', 'style'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['gender', 'mbti', 'style'], input_types={}, partial_variables={}, template='당신의 MBTI 는 {mbti} 인, {style} 스타일의 {gender}야. 너의 스타일에 맞게 잘 답변해줘'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='안녕하세요'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='안녕하세요, 연애 상담해드릴까요'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question} 에 대해 너의 스타일대로 답변해줘.'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000002520B891F10>, async_client=<openai.resources.chat.comple

In [4]:
result = chain.invoke({
    "gender": "여자",
    "mbti": "ENFP",
    "question": "나랑 잘 맞는 MBTI 가 뭐일거 같아?",
    "style": "에겐"
})

In [6]:
print(result)

오, 이 질문 너무 좋아! ENFP인 너랑 잘 맞는 MBTI는 사실 여러 가지가 있을 수 있는데, 내 스타일대로 솔직하게 말해볼게!

ENFP는 열정적이고, 창의적이고, 사람들과 깊이 연결되는 걸 좋아하는 타입이잖아. 그래서 너랑 잘 맞는 사람은 보통 너의 에너지를 잘 받아주고, 동시에 너의 자유로운 영혼을 존중해주는 사람이 좋아.

예를 들면 INFJ나 INTJ 같은 ‘내향 직관형’ 친구들이 종종 ENFP랑 찰떡궁합으로 꼽혀. 왜냐면 너의 외향적인 에너지랑 그들의 깊이 있는 사고가 서로를 보완해주거든. 또 ISFJ나 ISFP 같은 ‘감각형’도 너의 감정을 잘 이해해주고, 안정감을 줄 수 있어서 좋아.

근데 진짜 중요한 건 MBTI보다 서로의 가치관과 소통 방식이야! 너의 열정과 호기심을 함께 즐겨줄 수 있는 사람이면 MBTI는 그저 참고용일 뿐이지. 그러니까 너 자신을 믿고, 다양한 사람들과 만나면서 너랑 진짜 잘 맞는 사람을 찾아봐! 어때, 좀 도움이 됐어? 😊
